# Integrate a regression tree model to classify a person's position based on landmark distances
- Collect Training Data: Capture images with known position labels (e.g., "Standing", "Push-up Up", "Push-up Down", "Lying Down").
- Extract Features: Compute landmark distances (e.g., shoulder-to-hip, hip-to-knee, elbow angle, etc.).
- Train a Decision Tree: Use scikit-learn's DecisionTreeClassifier or DecisionTreeRegressor to classify or predict positions.
- Integrate into Real-Time Detection: Use the trained model to classify the current pose.

---
# Embeddings in a CSV file

---

In [1]:
import cv2
import mediapipe as mp
import csv
import numpy as np
import os

2025-04-01 16:19:54.306928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743517194.326154   36958 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743517194.332153   36958 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743517194.346455   36958 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743517194.346480   36958 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743517194.346482   36958 computation_placer.cc:177] computation placer alr

In [3]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Directory containing labeled folders with images
base_folder = "training_datasets/yoga_images"  # Change to your folder path

# Create CSV file and write headers
csv_filename = "models/yoga_landmark_positions.csv"
headers = ["shoulder_hip_dist", "hip_knee_dist", "elbow_angle", "shoulder_foot_dist", "hip_angle", "knee_angle", "label"]

with open(csv_filename, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    
    for label in os.listdir(base_folder):
        label_folder = os.path.join(base_folder, label)
        if not os.path.isdir(label_folder):
            continue
        
        for image_name in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_name)
            
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get key landmarks
                shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
                hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
                knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
                elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
                wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
                foot = landmarks[mp_pose.PoseLandmark.LEFT_HEEL]

                # Compute distances
                shoulder_hip_dist = np.linalg.norm([shoulder.x - hip.x, shoulder.y - hip.y])
                hip_knee_dist = np.linalg.norm([hip.x - knee.x, hip.y - knee.y])
                shoulder_foot_dist = np.linalg.norm([shoulder.x - foot.x, shoulder.y - foot.y])

                # Compute elbow angle
                def calculate_angle(a, b, c):
                    ba = np.array([a.x - b.x, a.y - b.y])
                    bc = np.array([c.x - b.x, c.y - b.y])
                    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
                
                elbow_angle = calculate_angle(shoulder, elbow, wrist)
                hip_angle = calculate_angle(shoulder, hip, knee)
                knee_angle = calculate_angle(hip, knee, foot)

                # Save to CSV
                writer.writerow([shoulder_hip_dist, hip_knee_dist, elbow_angle, \
                    shoulder_foot_dist, hip_angle, knee_angle, label])
                print( [shoulder_hip_dist, hip_knee_dist, elbow_angle, \
                    shoulder_foot_dist, hip_angle, knee_angle, label] )
                # print(f"Data saved: {shoulder_hip_dist}, {hip_knee_dist}, {elbow_angle}, {label}")
                            
            if results.pose_landmarks:
                h, w, _ = image.shape
                cv2.circle(image, (int(shoulder.x * w), int(shoulder.y * h)), 5, (0, 255, 0), -1) 
                cv2.circle(image, (int(hip.x * w), int(hip.y * h)), 5, (0, 255, 0), -1)  
                cv2.circle(image, (int(knee.x * w), int(knee.y * h)), 5, (0, 255, 0), -1)  
                cv2.circle(image, (int(shoulder.x * w), int(shoulder.y * h)), 5, (0, 255, 0), -1)  
                cv2.circle(image, (int(elbow.x * w), int(elbow.y * h)), 5, (0, 255, 0), -1)  
                cv2.circle(image, (int(foot.x * w), int(foot.y * h)), 5, (0, 255, 0), -1)  
                cv2.line(image, (int(hip.x * w), int(hip.y * h)), (int(shoulder.x * w), int(shoulder.y * h)), (255, 0, 255), 2)
                cv2.line(image, (int(hip.x * w), int(hip.y * h)), (int(knee.x * w), int(knee.y * h)), (255, 0, 255), 2)
            cv2.imshow("Pose Capture", image)

            if cv2.waitKey(500) & 0xFF == ord('q'):
                break

cv2.destroyAllWindows()

I0000 00:00:1743517503.940905   36958 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743517503.979596   37366 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
W0000 00:00:1743517504.082625   37360 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743517504.126722   37364 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x619ba885ae70) is not the object's thread (0x619ba990e570).
Cannot move to target thread (0x619ba885ae70)

QObject::moveToThread: Current thread (0x619ba885ae70) is not the object's thread (0x619ba990e570).
Cannot move to target thread (0x619ba885ae70)

QObject::moveToThread: Current thread (0x619ba885ae70) is not the object's thread (0

[0.41345594848991657, 0.2163011296379466, 149.84301352562858, 0.5121892990232713, 84.75392398082901, 3.6942512439050375, 'Padmasana']
[0.4446138470961082, 0.26179726364219197, 154.86287206967265, 0.5009543870120934, 101.96810326051033, 8.24376500358991, 'Padmasana']
[0.3687206421902611, 0.0929714377008622, 164.70772087415614, 0.39890623991862406, 109.56231813652344, 12.279613091560819, 'Padmasana']
[0.36025192314211, 0.22762267622415122, 167.4365890798184, 0.45121325309988275, 107.67062437706923, 15.239061289340407, 'Padmasana']
[0.4837150002171389, 0.17657320627867196, 175.58655900546364, 0.5390138553035239, 104.7567916347289, 13.96126353871561, 'Padmasana']
[0.41941891805465725, 0.2596573827226373, 126.87223842020259, 0.6005265242737473, 96.56320214202655, 21.66344442595517, 'Padmasana']
[0.43540719918196386, 0.43060068249775146, 115.06959169612429, 0.4424054037384987, 89.65432889078072, 11.858237718796802, 'Padmasana']
[0.4695230155852367, 0.22791686601873246, 134.62562891128752, 0.

---
# Classification

---

In [4]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import joblib

In [5]:
# Load collected data
data = pd.read_csv("models/yoga_landmark_positions.csv")  # Columns: distances + labels
data

,shoulder_hip_dist,hip_knee_dist,elbow_angle,shoulder_foot_dist,hip_angle,knee_angle,label
0,0.413456,0.216301,149.843014,0.512189,84.753924,3.694251,Padmasana
1,0.444614,0.261797,154.862872,0.500954,101.968103,8.243765,Padmasana
2,0.368721,0.092971,164.707721,0.398906,109.562318,12.279613,Padmasana
3,0.360252,0.227623,167.436589,0.451213,107.670624,15.239061,Padmasana
4,0.483715,0.176573,175.586559,0.539014,104.756792,13.961264,Padmasana
...,...,...,...,...,...,...,...
246,0.312882,0.191965,170.745583,0.510284,110.197709,144.301730,Hanumanasana
247,0.478600,0.222851,163.206541,0.562277,113.804006,109.878318,Hanumanasana
248,0.391930,0.184343,177.903845,0.584975,98.712225,173.372061,Hanumanasana
249,0.392888,0.220667,174.146112,0.580734,115.126194,148.665294,Hanumanasana


In [6]:
X = data.drop(columns=['label'])  # Features: distances
y = data['label']  # Target: position labels

# Train classifier
clf = DecisionTreeClassifier()
clf.fit(X, y)

# Save model
joblib.dump(clf, "models/yoga_classifier.pkl")

['models/yoga_classifier.pkl']